In [1]:
import pandas as pd
from pandas import Series, DataFrame
import math
%matplotlib inline
import numpy as np
import torch
from torch import nn
from d2l import torch as d2l

In [2]:
#读取原始数据
test = pd.read_csv("../data/220617_1855_ethanol10ppm.csv")
# 第八行无名称的数据改成pulse
test.rename(columns={'Unnamed: 8': 'pulse'},inplace=True)


In [3]:
origin_data_length = 90
traget_data_length = 70
all = test
count = math.floor(len(test)/origin_data_length) # 筛选的一个周期为90个数据
zero = [0]*count

In [4]:
for j in range(count):
    
    for i in range (j*origin_data_length,(j+1)*origin_data_length):
        if test.loc[i,"pulse"] > 2.5 and test.loc[(i-1),"pulse"] < 0.5: #根据脉冲的特性找到跳变开始的点
            zero[j] = test.index[i]

In [5]:
for i in zero:
    p = zero.index(i)
    #print(p)
    for j in range(traget_data_length):  # traget_data_length 是脉冲前20个点 脉冲过程30个  脉冲后20个
        all.iloc[p*traget_data_length+j,:] = test.iloc[i-20+j,:]

KeyboardInterrupt: 

In [ ]:
all.to_csv('../data/allout.csv') 

In [ ]:
data_set =pd.read_csv ('../data/allout.csv')

In [ ]:
redundancy = all.shape[0]%traget_data_length

In [ ]:
all_feature = all.iloc[0:-redundancy,1:]

In [ ]:
all_feature.head()

In [ ]:
sensor1 = all_feature.iloc[:,0]
sensor2 = all_feature.iloc[:,1]
sensor3 = all_feature.iloc[:,2]
sensor4 = all_feature.iloc[:,3]

In [ ]:

train_features = torch.Tensor(sensor1.values)
true_train_features1 = train_features.reshape(-1,traget_data_length)
train_features = torch.Tensor(sensor2.values)
true_train_features2 = train_features.reshape(-1,traget_data_length)
train_features = torch.Tensor(sensor3.values)
true_train_features3 = train_features.reshape(-1,traget_data_length)
train_features = torch.Tensor(sensor4.values)
true_train_features4 = train_features.reshape(-1,traget_data_length)

In [ ]:
label = all_feature.iloc[:,6]
train_label = torch.tensor(label.values)
true_train_label = train_label.reshape(-1,traget_data_length)

In [ ]:
a = true_train_features1.detach().numpy()
b = true_train_features2.detach().numpy()
c = true_train_features3.detach().numpy()
d = true_train_features4.detach().numpy()
l = true_train_label.detach().numpy()

In [ ]:
a1 = pd.DataFrame(a)
b1 = pd.DataFrame(b)
c1 = pd.DataFrame(c)
d1 = pd.DataFrame(d)
l1 = pd.DataFrame(l)

In [ ]:
a1.loc[:,traget_data_length] = l1.iloc[:,0]
b1.loc[:,traget_data_length] = l1.iloc[:,0]
c1.loc[:,traget_data_length] = l1.iloc[:,0]
d1.loc[:,traget_data_length] = l1.iloc[:,0]

In [ ]:
a1.to_csv('../data/sensor_data/sensor1_1ppm.csv',index=0)
b1.to_csv('../data/sensor_data/sensor2_1ppm.csv',index=0)
c1.to_csv('../data/sensor_data/sensor3_1ppm.csv',index=0)
d1.to_csv('../data/sensor_data/sensor4_1ppm.csv',index=0)

# 不同浓度采样

In [6]:
read_data_1ppm = pd.read_csv('../data/sensor1_with_label_1ppm_eth.csv',index_col=0)
read_data_5ppm = pd.read_csv('../data/sensor1_with_label_5ppm_eth.csv',index_col=0)
read_data_10ppm = pd.read_csv('../data/sensor1_with_label_10ppm_eth.csv',index_col=0)  #  需要修改读取的文件

In [7]:
train_data_1ppm = read_data_1ppm.iloc[2000:3000,0:-1]
label_data_1ppm = read_data_1ppm.iloc[2000:3000,-1]
train_data_5ppm = read_data_5ppm.iloc[2000:3000,0:-1]
label_data_5ppm = read_data_5ppm.iloc[2000:3000,-1]
train_data_10ppm = read_data_10ppm.iloc[2000:3000,0:-1]
label_data_10ppm = read_data_10ppm.iloc[2000:3000,-1]

In [8]:
#添加label列  0 1 2 softmax特有的
train_data_1ppm.loc[:,71]=0
train_data_5ppm.loc[:,71]=1
train_data_10ppm.loc[:,71]=2

In [9]:
test_data_noppm = pd.concat([train_data_1ppm,train_data_5ppm,train_data_10ppm],axis=0)

In [12]:
#随机采样 生成学习数据
sample = test_data_noppm.sample(100,replace=False,random_state=42,axis=0)
sample.to_csv("../data/test.csv",index=0)